In [ ]:
# This notebook performs telescope motion tests in elevation and looks at how the mirror position changes as per the measurements by the PMD

In [1]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

from lsst.ts.observatory.control.utils import RotType
import logging

In [2]:
# for tab completion to work
%config IPCompleter.use_jedi = False

In [3]:
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [4]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

file:///home/patrickingraham/develop/ts_ddsconfig/config/ospl-shmem.xml
summit


In [5]:
domain = salobj.Domain()

In [6]:
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.01 sec
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, algorithm)
Read 4 history items for RemoteEvent(ATDomeTrajectory, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, authList)
Read 3 history items for RemoteEvent(ATDomeTrajectory, 0, followingMode)
Read 100 history items for RemoteEvent(ATDomeTrajectory, 0, heartbeat)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, logLevel)
Read 17 history items for RemoteEvent(ATDomeTrajectory, 0, logMessage)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, settingVersions)
Read 1 history ite

[[None, None, None, None, None, None, None], [None, None, None, None]]

timeAndDate DDS read queue is full (100 elements); data may be lost
m1AirPressure DDS read queue is filling: 18 of 100 elements
mountPositions DDS read queue is filling: 23 of 100 elements
loadCell DDS read queue is filling: 18 of 100 elements


In [7]:
await atcs.enable()

Enabling all components
Gathering settings.
Couldn't get settingVersions event. Using empty settings.
Complete settings for atmcs.
Complete settings for atptg.
Complete settings for ataos.
Complete settings for atpneumatics.
Complete settings for athexapod.
Complete settings for atdome.
Complete settings for atdometrajectory.
Settings versions: {'atmcs': '', 'atptg': '', 'ataos': 'current', 'atpneumatics': '', 'athexapod': 'current', 'atdome': 'current', 'atdometrajectory': ''}
[atmcs]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atptg]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[ataos]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atpneumatics]::[<State.ENABLED: 2>]
[athexapod]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atdome]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atdometrajectory]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [8]:
atcs.check.atdome=False
atcs.check.atdometrajectory=False

In [9]:
tmp=await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply='', timeout=30)
print(tmp)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]


In [10]:
tmp=await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY, settingsToApply='', timeout=30)
print(tmp)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]


### Connect to PMDs
#### Bring online if required

In [11]:
pmd_remote = salobj.Remote(name="PMD", domain=domain, index=1)
await pmd_remote.start_task

Read historical data in 0.00 sec
Read 4 history items for RemoteEvent(PMD, 1, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(PMD, 1, authList)
Read 100 history items for RemoteEvent(PMD, 1, heartbeat)
Read 4 history items for RemoteEvent(PMD, 1, logLevel)
Read 100 history items for RemoteEvent(PMD, 1, logMessage)
Read 1 history items for RemoteEvent(PMD, 1, metadata)
Read 1 history items for RemoteEvent(PMD, 1, settingVersions)
Read 1 history items for RemoteEvent(PMD, 1, settingsApplied)
Read 1 history items for RemoteEvent(PMD, 1, simulationMode)
Read 1 history items for RemoteEvent(PMD, 1, softwareVersions)
Read 15 history items for RemoteEvent(PMD, 1, summaryState)


In [12]:
tmp = await pmd_remote.evt_heartbeat.next(flush=True,timeout=15)
print(tmp)

PMDID: 1, private_revCode: f6854b27, private_sndStamp: 1626293254.3074937, private_rcvStamp: 1626293254.308333, private_seqNum: 504448, private_identity: PMD:1, private_origin: 7310, private_host: 0, heartbeat: False, priority: 0


In [13]:
tmp = await pmd_remote.cmd_setLogLevel.set_start(level=10)

In [14]:
tmp = await pmd_remote.evt_summaryState.aget(timeout=15)
print(salobj.State(tmp.summaryState))

State.ENABLED


In [ ]:
tmp=await salobj.set_summary_state(pmd_remote, salobj.State.ENABLED, settingsToApply='micrometer', timeout=30)
print(tmp)

In [15]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

PMDID: 1, private_revCode: 819e90fa, private_sndStamp: 1626293271.0435886, private_rcvStamp: 1626293271.044803, private_seqNum: 106882, private_identity: PMD:1, private_origin: 7310, private_host: 0, position: [-1.077, 0.423, -21.771, 11.852, 5.128, nan, nan, nan]


### Setup the script to write log messages
#### note that the index cannot be used anywhere else!

In [16]:
#start the controller to write script messages and grab the domain
c = salobj.Controller(name="Script", index=os.getuid())

In [17]:
c.log.setLevel(logging.DEBUG)

Read historical data in 0.02 sec


In [19]:
c.log.info('Patrick Testing INFO level ') # only 

Patrick Testing INFO level 


In [ ]:
az=0; el=80
c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}')

In [ ]:
el_samples1 = np.arange(15,83,8)
el_samples2 = np.arange(83,15,-8)

In [ ]:
print(el_samples1)
print(el_samples2)

In [ ]:
el_samples=np.concatenate((el_samples1, el_samples2))

In [ ]:
el_samples

In [ ]:
atcs.check.atdome=False
atcs.check.atdometrajectory=False

In [ ]:
print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

In [ ]:
az=0
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')

In [ ]:
await atcs.point_azel(az=az, el=82)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=116000)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=130000)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=135000)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=135000)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=140000)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)
await atcs.point_azel(az=az, el=50)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)
await asyncio.sleep(4)
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
# now craig will push on it

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
import datetime
datetime.

In [ ]:
az=0
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')

In [ ]:
c.log.info('Previous dataset was after craig tweaked the gauge') # only 

In [ ]:
c.log.info('Previous dataset had no active optics system enabled') # only 


In [ ]:
await atcs.point_azel(az=az, el=80)

In [ ]:
await atcs.stop_tracking()

In [ ]:
await atcs.rem.ataos.cmd_enable.start()

In [ ]:
await atcs.rem.atpneumatics.cmd_m1CloseAirValve.start()

In [ ]:
await atcs.rem.atpneumatics.cmd_closeMasterAirSupply.start()

In [ ]:
await atcs.rem.atpneumatics.cmd_closeInstrumentAirValve.start()

In [ ]:
await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True)

In [ ]:
az=0
c.log.info('Starting run with m1 correction enabled')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}')
    

In [ ]:
await atcs.point_azel(az=0, el=80)

In [ ]:
await atcs.stop_tracking()

In [ ]:
atcs.check.atdome=True
await atcs.standby()

In [ ]:
tmp=await salobj.set_summary_state(pmd_remote, salobj.State.OFFLINE, settingsToApply='micrometer', timeout=30)
print(tmp)